In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Text, \
    select, update, func
from tqdm.notebook import tqdm
import praw
import time

In [2]:
subreddit = 'askreddit'
db_name = '%s.db' % subreddit
seed = 0
stop_at = 100000
exclude_top_level = True

In [3]:
engine = create_engine('sqlite:///%s' % db_name, echo = False)
meta = MetaData()

submissions_table = Table(
   'submissions', meta, 
    Column('id', String, primary_key=True),
    Column('status', Integer),
)

In [4]:
meta = MetaData()
comments_table = Table(
   'comments', meta, 
    Column('comment_id', String, primary_key=True),
    Column('submission_id', String),
    Column('url', String),
    Column('comment', Text),    
)
meta.create_all(engine)

In [5]:
reddit = praw.Reddit(
    client_id="PTYaeRF8ATlqboPxx-3u6g",
    client_secret="o9GvtpP9xk-CXfj7Y5Tys62dOZSyRw",
    user_agent="comment extraction for prevalence estimation (by u/patroklos1)",
    #password="8PmSSKqNfnQ3hvy",
    #username="patroklos1",
)

In [6]:
with engine.connect() as conn:
    total_rows = engine.execute('select count(id) from submissions').scalar()
    print(total_rows)
    collected_rows = engine.execute('select count(id) from submissions where status=0').scalar()
    print(collected_rows)
    
    print(1 - collected_rows / total_rows)

/tmp/ipykernel_15504/2453679325.py:2: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  total_rows = engine.execute('select count(id) from submissions').scalar()


283531
277682
0.0206291375546237


In [7]:
with engine.connect() as conn:
    total_rows = engine.execute('select count(comment_id) from comments').scalar()
    print(total_rows)

43736


In [ ]:
while True:
    with engine.connect() as conn:
        num_rows = engine.execute('select count(comment_id) from comments').scalar()
        
    print(num_rows)
    if num_rows >= stop_at:
        break
    
    with engine.connect() as conn:
        # get an unvisited submission
        stmt = submissions_table.select()
        stmt = stmt.where(submissions_table.c.status==0).order_by(func.random()).limit(1)
        row = next(conn.execute(stmt))
        submission_id, status = row

        # update
        stmt = submissions_table.update()
        stmt = stmt.where(submissions_table.c.id==submission_id)
        stmt = stmt.values(status=1)
        conn.execute(stmt)

    # retrieve comments
    submission = reddit.submission(submission_id)
    submission.comments.replace_more(limit=None)
    data = []
    for comment in submission.comments.list():
        url = "https://www.reddit.com/r/%s/comments/%s/comment/%s" % (subreddit, submission_id, comment.id)
        
        if exclude_top_level and comment.parent_id.startswith('t3'):
            continue
        
        data.append((comment.id, submission_id, url, comment.body))

    # save
    with engine.connect() as conn:
        # update
        stmt = submissions_table.update()
        stmt = stmt.where(submissions_table.c.id==submission_id)
        stmt = stmt.values(status=2)
        conn.execute(stmt)

        if len(data) == 0:
            continue
        else:
            for id, submission_id, url, comment in data:
                ins = comments_table.insert().values(
                    comment_id=id, submission_id=submission_id, url=url, comment=comment)
                result = conn.execute(ins)
        
    # time.sleep(0.1)

43736
43736
43736
43736
43738
43738
43738
43738
43739
43739
43739
43739
43740
43740
43740
43740
43741
43741
43741
43741
43779
43779
43779
43784
43786
43786
43786
43787
43787
43788
43788
43788
43788
43788
43788
43788
43790
43803
43803
43803
43803
43823
43823
43823
43823
43824
43825
43825
43826
43826
43826
43826
43826
43826
43826
43827
43828
43828
43832
43832
43832
43832
43834
43834
43834
43834
43834
43836
43836
43839
43842
43846
43846
43846
43846
43846
43848
43848
43854
43857
43862
43862
43862
43862
43874
43876
43876
43876
43876
43876
43877
43878
43879
43879
43880
43881
43887
43888
43888
43892
43892
43895
43895
43895
43895
43945
43945
43946
43946
43952
43952
43954
43954
44236
44236
44236
44236
44236
44236
44236
44244
44244
44244
44244
44245
44245
44245
44247
44250
44254
44254
44254
44261
44261
44262
44262
44263
44263
44263
44263


In [ ]:
with engine.connect() as conn:
    stmt = comments_table.select()
    stmt = stmt.order_by(func.random()).limit(1)
    row = next(conn.execute(stmt))
    print(row)